# Doppler Solve: Bilinear

## Setup

In [ ]:
%matplotlib inline

In [ ]:
%run notebook_setup.py

In [ ]:
import starry

starry.config.lazy = False
starry.config.quiet = True

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import starry
import george

In [ ]:
def generate(
    nc=1,
    show=True,
    flux_err=1e-4,
    ydeg=15,
    u=[0.5, 0.25],
    nt=16,
    inc=40,
    veq=60000,
    smoothing=0.075,
    **kwargs
):

    # Instantiate
    wav = np.linspace(642.85, 643.15, 200)
    map = starry.DopplerMap(
        lazy=False,
        ydeg=ydeg,
        udeg=len(u),
        nc=nc,
        veq=veq,
        inc=inc,
        nt=nt,
        wav=wav,
    )

    # Limb darkening (TODO: fix __setitem__)
    map._u = np.append([-1.0], u)

    # Component surface images
    if nc == 1:
        images = ["spot"]
    elif nc == 2:
        images = ["star", "spot"]
    else:
        raise NotImplementedError("")

    # Component spectra
    if nc == 1:
        mu = np.array([643.0])
    elif nc == 2:
        mu = np.array([643.025, 642.975])
    else:
        raise NotImplementedError("")
    sig = 0.0085
    dw = map.wav0.reshape(1, -1) - mu.reshape(-1, 1)
    spectra = (1.0 - np.exp(-0.5 * dw ** 2 / sig ** 2))[:nc]

    # Load the component maps
    map.load(maps=images, spectra=spectra, smoothing=smoothing)

    # Show
    if show:
        map.show_components(show_spectra=True)

    # Generate unnormalized data
    flux0 = map.flux(normalize=False)
    flux0 += flux_err * np.random.randn(*flux0.shape)

    # Generate normalized data
    flux = map.flux(normalize=True)
    flux += flux_err * np.random.randn(*flux.shape)

    # Reset the map
    def reset_spectrum():
        map._spectrum = np.ones((map.nc, map.nw0_))

    def reset_map():
        map._y = np.zeros((map.Ny, map.nc))
        map._y[0] = 1.0

    map.reset_spectrum = reset_spectrum
    map.reset_map = reset_map

    return map, flux0, flux

## Solve for the spectrum

### One component

#### Generate the data

In [ ]:
settings = dict(
    flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075
)
map, flux0, flux = generate(nc=1, **settings)

#### Unnormalized solve

In [ ]:
map.reset_spectrum()
map.solve(flux0, normalized=False, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

#### Normalized solve

In [ ]:
map.reset_spectrum()
map.solve(flux, normalized=True, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

### Two components

#### Generate the data

In [ ]:
settings = dict(
    flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075
)
map, flux0, flux = generate(nc=2, **settings)

#### Unnormalized solve

In [ ]:
map.reset_spectrum()
map.solve(flux0, normalized=False, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

#### Normalized solve

In [ ]:
map.reset_spectrum()
map.solve(flux, normalized=True, fix_map=True, flux_err=settings["flux_err"])
map.show_components(show_maps=False)

## Solve for the map

### One component

#### Generate the data

In [ ]:
settings = dict(
    flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075
)
map, flux0, flux = generate(nc=1, **settings)

#### Unnormalized solve

In [ ]:
map.reset_map()
map.solve(
    flux0, normalized=False, fix_spectrum=True, flux_err=settings["flux_err"]
)
map.show_components(show_spectra=False)

#### Normalized solve w/ tempering

In [ ]:
map.reset_map()
map.solve(
    flux,
    normalized=True,
    fix_spectrum=True,
    flux_err=settings["flux_err"],
    logT0=12,
)
map.show_components(show_spectra=False)

### Two components

#### Generate the data

In [ ]:
settings = dict(
    flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075
)
map, flux0, flux = generate(nc=2, **settings)

#### Unnormalized solve

In [ ]:
map.reset_map()
map.solve(
    flux0, normalized=False, fix_spectrum=True, flux_err=settings["flux_err"]
)
map.show_components(show_spectra=False)

#### Normalized solve w/ tempering

In [ ]:
map.reset_map()
map.solve(
    flux,
    normalized=True,
    fix_spectrum=True,
    flux_err=settings["flux_err"],
    logT0=12,
)
map.show_components(show_spectra=False)

## Solve for everything

### One component

#### Generate the data

In [ ]:
settings = dict(
    flux_err=1e-4, ydeg=15, nt=16, inc=40, veq=60000, smoothing=0.075
)
map, flux0, flux = generate(nc=1, **settings)

#### Unnormalized solve

In [ ]:
spectral_cov = george.GP(
    1e-2 * george.kernels.Matern32Kernel(1e-3)
).get_matrix(map.wav0)
info = map.solve(
    flux0,
    normalized=False,
    flux_err=settings["flux_err"],
    logT0=2,
    spectral_cov=spectral_cov,
)
map.show_components()

#### Normalized solve

In [ ]:
spectral_cov = george.GP(
    1e-2 * george.kernels.Matern32Kernel(1e-3)
).get_matrix(map.wav0)
map.solve(
    flux,
    normalized=True,
    flux_err=settings["flux_err"],
    logT0=2,
    spectral_cov=spectral_cov,
)
map.show_components()

### Two components

This is in general very hard to do without good prior information! See the separate notebook on the general two-component problem.